# 设置

以下单元格从 `.env` 文件中读取 OpenAI API 密钥：

In [4]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

在整个课程中，我们将使用OpenAI的`gpt-3.5-turbo`模型和[聊天完成端点](https://platform.openai.com/docs/guides/chat)。

这个辅助函数将使得使用提示和查看生成的输出更加容易：

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 这是模型输出的随机程度
    )
    return response.choices[0].message["content"]

## 提示原则
- **原则1：编写清晰明确的指令**
- **原则2：给模型足够的“思考”时间**

### 原则1：编写清晰明确的指令

#### 策略1：使用分隔符清晰地指示输入的不同部分
- 分隔符可以是任何东西，如：```, """, < >, `<tag> </tag>`, `:`

In [7]:
text = f"""
您应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的任务。\
这将引导模型朝着期望的输出方向发展，并减少接收到无关或不正确响应的可能性。\
不要将编写清晰提示与编写简短提示混淆。\
在许多情况下，较长的提示提供更多的清晰度和上下文，这可以导致更详细和相关的输出。
"""
prompt = f"""
将三个反引号分隔的文本总结为一句话。
```{text}```
"""
response = get_completion(prompt)
print(response)

编写清晰、具体的指令可以引导模型朝着期望的输出方向发展，减少无关或不正确响应的可能性，较长的提示提供更多的清晰度和上下文，可以导致更详细和相关的输出。


#### 策略2：要求结构化输出
- JSON，HTML

In [9]:
prompt = f"""
生成三个虚构书名，包括它们的作者和类型。\
以JSON格式提供，包括以下键：\
book_id，title，author，genre
"""
response = get_completion(prompt)
print(response)

{
  "book1": {
    "book_id": 1,
    "title": "The Lost City",
    "author": "Emily Jones",
    "genre": "Adventure"
  },
  "book2": {
    "book_id": 2,
    "title": "The Secret Garden",
    "author": "Sophie Lee",
    "genre": "Fantasy"
  },
  "book3": {
    "book_id": 3,
    "title": "The Last Hope",
    "author": "David Chen",
    "genre": "Science Fiction"
  }
}


In [10]:
text_1 = f"""
泡一杯茶很容易！首先，你需要把一些水烧开。在这个过程中，\
拿一个杯子并放入一个茶包。一旦水足够热，\
只需将其倒在茶包上。让它浸泡一会儿。几分钟后，\
取出茶包。如果你喜欢，可以加一些糖或牛奶调味。\
就这样！你有一杯美味的茶可以享用了。
"""

prompt = f"""
您将获得由三个引号分隔的文本。如果它包含一系列指令，\
请按以下格式重新编写这些指令：

第1步 - 需要把一些水烧开。
第2步 - 拿一个杯子并放入一个茶包。
第3步 - 将热水倒在茶包上。
第4步 - 让茶包浸泡一会儿。
第5步 - 取出茶包。
第6步 - 如果需要，加入糖或牛奶调味。
第7步 - 完成！享用您的美味茶。

如果文本不包含一系列指令，则只需写“未提供步骤。”

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1的完成情况：")
print(response)

Text 1的完成情况：
第1步 - 需要把一些水烧开。
第2步 - 拿一个杯子并放入一个茶包。
第3步 - 将热水倒在茶包上。
第4步 - 让茶包浸泡一会儿。
第5步 - 取出茶包。
第6步 - 如果需要，加入糖或牛奶调味。
第7步 - 完成！享用您的美味茶。


<block>
策略3：要求模型检查条件是否满足
</block>

In [12]:
text_2 = f"""
今天阳光明媚，鸟儿在唱歌。这是一个去公园散步的美好日子。花儿盛开，树木在微风中轻轻摇曳。人们出门享受美好的天气。有些人在野餐，有些人在玩游戏，\
还有些人只是在草地上放松。这是一个完美的日子，可以在户外度过时间，欣赏大自然的美丽。
"""

prompt = f"""
您将获得由三个引号分隔的文本。如果它包含一系列指令，\ 
请按以下格式重新编写这些指令：

第1步 - ...
第2步 - …
…
第N步 - …

如果文本不包含一系列指令，则只需写“未提供步骤。”

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
未提供步骤。


#### 策略4：“少样本”提示

In [13]:
prompt = f"""
你的任务是以一致的风格回答。

<child>: 教我耐心。

<grandparent>: 刻出最深的峡谷的河流源于一眼泉水； 最宏伟的交响乐始于一支乐曲； 最复杂的挂毯始于一根孤独的线。

<child>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<grandparent>: 韧性来自于坚定的信念和不屈的意志，就像一棵树在风雨中弯曲但不折断。要有耐心和毅力，不断努力，才能在生活中展现出真正的韧性。


### 原则2：给模型时间“思考”

#### 策略1：明确完成任务所需的步骤

In [14]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
prompt_1 = f"""
执行以下操作：
1-用1句话总结三重反引号分隔的以下文本。
2-将摘要翻译成中文。
3-列出中文摘要中的每个名称。
4-输出一个包含以下键的json对象：chinese_summary，num_names。

用换行符分隔您的答案。

文本：
```{text}```
"""
response = get_completion(prompt_1)
print("Prompt 1的完成情况：")
print(response)

Prompt 1的完成情况：
1- Siblings Jack and Jill go on a quest for water, but misfortune strikes. Despite this, they remain adventurous and continue exploring.
2- 在一个迷人的村庄里，兄妹杰克和吉尔出发去山顶的井里取水。当他们欢快地唱着歌爬上去时，不幸降临了——杰克绊倒在一块石头上，滚下了山坡，吉尔也跟着摔了下来。虽然有些受伤，但他们还是回到了家里得到了安慰的拥抱。尽管发生了不幸，他们的冒险精神仍然不减，他们继续愉快地探索。
3- Jack、Jill
4- {"chinese_summary": "在一个迷人的村庄里，兄妹杰克和吉尔出发去山顶的井里取水。当他们欢快地唱着歌爬上去时，不幸降临了——杰克绊倒在一块石头上，滚下了山坡，吉尔也跟着摔了下来。虽然有些受伤，但他们还是回到了家里得到了安慰的拥抱。尽管发生了不幸，他们的冒险精神仍然不减，他们继续愉快地探索。", "num_names": 2}


#### 请求以指定格式输出

In [15]:
prompt_2 = f"""
你的任务是执行以下操作：
1-用1个句子总结以下用<>分隔的文本。
2-将摘要翻译成中文。
3-列出中文摘要中的每个名称。
4-输出一个包含以下键的json对象：chinese_summary，num_names。

使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要翻译>
名称：<中文摘要中的名称列表>
输出JSON：<具有摘要和num_names的json>

文本：<{text}>
"""
response = get_completion(prompt_2)
print("\nPrompt 2的完成情况：")
print(response)


Prompt 2的完成情况：
摘要：Jack和Jill在一个迷人的村庄里出发去山顶的井里取水，但不幸的是，Jack绊倒了，Jill也跟着滚下山。尽管有些受伤，但他们的冒险精神仍然不减，继续愉快地探索。

名称：Jack，Jill

输出JSON：{"chinese_summary": "Jack和Jill在一个迷人的村庄里出发去山顶的井里取水，但不幸的是，Jack绊倒了，Jill也跟着滚下山。尽管有些受伤，但他们的冒险精神仍然不减，继续愉快地探索。", "num_names": 2}


<block>
策略2：在匆忙得出结论之前，指导模型自行解决问题。
</block>

In [16]:
prompt = f"""
确定学生的解决方案是否正确。

问题：
我正在建造一个太阳能发电装置，我需要帮助计算财务。\
- 土地成本为每平方英尺100美元\
- 我可以购买每平方英尺250美元的太阳能电池板\
- 我谈判了一份维护合同，每年将花费我固定的10万美元，以及每平方英尺额外的10美元\
作为第一年运营的总成本，其函数为平方英尺的数量。

学生的解决方案：
设x为安装面积，单位为平方英尺。
成本：
1. 土地成本：100x
2. 太阳能电池板成本：250x
3. 维护成本：100,000 + 100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

这个解决方案是正确的，它考虑了土地成本、太阳能电池板成本和维护成本，并计算出了总成本。


注意，学生的解决方案实际上是不正确的。
我们可以通过指示模型先找出自己的解决方案来解决这个问题。

In [18]:
prompt = f"""
你的任务是确定学生的解决方案是否正确。
解决问题的步骤如下：
- 首先，自己解决问题。
- 然后将自己的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己解决问题之前，不要决定学生的解决方案是否正确。

使用以下格式：
问题：
```
问题在这里
```
学生的解决方案：
```
学生的解决方案在这里
```
实际解决方案：
```
解决方案和你的解决方案的步骤在这里
```
学生的解决方案是否与实际解决方案相同：
```
是或否
```
学生的成绩：
```
正确或不正确
```

问题：
```
我正在建造一个太阳能发电装置，我需要帮助解决财务问题。
- 土地成本为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我谈判了一份维护合同，每年的费用为固定的10万美元，以及每平方英尺额外的10美元
作为平方英尺数量的函数，第一年运营的总成本是多少。
``` 
学生的解决方案：
```
设x为安装面积的大小。
成本：
1. 土地成本：100x
2. 太阳能电池板成本：250x
3. 维护成本：100,000 + 100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
```
实际解决方案：
"""
response = get_completion(prompt)
print(response)

解决方案和你的解决方案的步骤在这里：
学生的解决方案是正确的。

第一年的总成本为：
1. 土地成本：100 * x 平方英尺
2. 太阳能电池板成本：250 * x 平方英尺
3. 维护成本：100,000 + 10 * x 平方英尺
总成本：100 * x + 250 * x + 100,000 + 10 * x = 360 * x + 100,000

学生的解决方案是否与实际解决方案相同：
是

学生的成绩：
正确


## 模型限制：幻觉
- Boie是一家真实的公司，产品名称不是真实的。

In [19]:
prompt = f"""
告诉我关于Boie的AeroGlide UltraSlim智能牙刷的信息
"""
response = get_completion(prompt)
print(response)

Boie的AeroGlide UltraSlim智能牙刷是一款高科技的电动牙刷，采用了先进的声波技术，能够提供高效的清洁效果。该牙刷的刷头非常细小，能够轻松进入牙缝和牙龈，清洁效果非常出色。此外，该牙刷还配备了智能感应技术，能够自动调节清洁力度和时间，让用户更加方便地使用。该牙刷还采用了环保材料制造，非常安全健康。总之，Boie的AeroGlide UltraSlim智能牙刷是一款高品质、高效率、高科技的电动牙刷，非常适合现代人的口腔清洁需求。
